In [1]:
from datasets import load_dataset
import re

# Load the dataset

In [2]:
dataset = load_dataset('timdettmers/openassistant-guanaco')

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
len(dataset)

2

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

# Shuffle the dataset and slice it

In [5]:
dataset = dataset['train'].shuffle(seed = 42).select(range(1000))

In [6]:
len(dataset)

1000

In [7]:
dataset[0]

{'text': '### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.'}

In [8]:
dataset[0]['text']

'### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.'

In [9]:
test_split = dataset[0]['text'].split('###')
test_split

['',
 ' Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?',
 ' Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.']

# LLAMA2 Instruction Prompt Format.

In [42]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "workspace/llama2_finetuning_tutorial/llama2_instruction_prompt.png")

# Function to transform the dataset.

In [39]:
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')
    # print(segments)
    # print(len(segments))
    

    reformatted_segments = []
    
    # Iterate over pairs of segments
    # start from 1 and get to len(segment) - 1, because the last conversation needs to be in a pair of - human and assistant
    # Jump in pair of 2 - because the conversation is in the form of 'human', 'assistant'
    for i in range(1, len(segments) - 1, 2 ):
        # print(segments[i])
        # print(segments[i].strip()) # remove the preceding space
        # print(segments[i].strip().replace('Human:','')) # replace the Human with a space
        # print(segments[i].strip().replace('Human:','').strip()) # remove the preceding space
        human_text = segments[i].strip().replace('Human:','').strip()

        # Check if any assistant is present
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:','').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text}</s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    # print(reformatted_segments)

    # Combine the Instruction prompt
    text = {'text': "".join(reformatted_segments)}
    print(text)
    print("\n")

    return text

        

In [40]:
transformed_dataset = dataset.map(transform_conversation)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.</s>'}


{'text': '<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии величия человек

In [41]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})